# Extracting insights from structured data with Text to SQL
In this notebook we will explore another important modality of data, structured data, and how we can leverage it to extract insights from it. We will prompt the LLM to translate a user's question into a SQL query that runs against a database. Text to SQL is a task where the goal is to convert a natural language question into a SQL query that can be executed on a database to retrieve the answer. This task is particularly useful in the context of databases, where users can ask questions in natural language and get the answers in a structured format.

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment("sql_rag_requirements.txt")

In [ ]:
required_models = [
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
]
validate_model_access(required_models)

In [ ]:
# delete any prior artifacts
!rm -f chroma.sqlite3 credit_scoring.duckdb 

## Data Ingestion
The first step is to obtain the dataset and load it into a database for querying. We'll download an [OpenML credit loans](https://www.openml.org/search?type=data&status=active&id=45938&sort=runs) and then load it into a [DuckDB database](https://duckdb.org/). 

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd

In [ ]:
data = fetch_openml(data_id=45938)["data"]

In [ ]:
# preview the dataset
data.head()

The dataset comes as a single table which contains the customer, accounts, and monthly payment information. To make this more realistic, we will break the dataset into 3 tables one for each entity (customer, account, and monthly payment) and then create a foreign key relationship between them.

In [ ]:
def extract_customers(df):
    customers = (
        df[["Customer_ID", "Name", "Age", "SSN", "Occupation", "Annual_Income"]]
        .drop_duplicates("Customer_ID")
        .reset_index(drop=True)
    )
    return customers


def extract_accounts(df):
    accounts = (
        df[
            [
                "Customer_ID",
                "Num_Bank_Accounts",
                "Num_Credit_Card",
                "Num_of_Loan",
                "Type_of_Loan",
            ]
        ]
        .drop_duplicates("Customer_ID")
        .reset_index(drop=True)
    )
    return accounts


def extract_monthly_records(df):
    monthly_records = df[
        [
            "ID",
            "Customer_ID",
            "Month",
            "Monthly_Inhand_Salary",
            "Delay_from_due_date",
            "Num_of_Delayed_Payment",
            "Changed_Credit_Limit",
            "Num_Credit_Inquiries",
            "Credit_Mix",
            "Outstanding_Debt",
            "Credit_Utilization_Ratio",
            "Credit_History_Age",
            "Payment_of_Min_Amount",
            "Total_EMI_per_month",
            "Amount_invested_monthly",
            "Payment_Behaviour",
            "Monthly_Balance",
        ]
    ]
    return monthly_records


customers_table = extract_customers(data)
accounts_table = extract_accounts(data)
monthly_records_table = extract_monthly_records(data)

Now that we have the dataset split into 3 tables, we can create a database and load the data into it. We will use DuckDB, a lightweight in-memory database that is easy to use and has good support for SQL queries.

We do this by running DDL commands and ingesting the pandas dataframes into the database.

In [ ]:
import duckdb


# Create or connect to a DuckDB database
conn = duckdb.connect("credit_scoring.duckdb")

# Create tables and insert data into DuckDB

# Customers Table
customers_ddl = "CREATE TABLE IF NOT EXISTS customers (Customer_ID VARCHAR, Name VARCHAR, Age INTEGER, SSN VARCHAR, Occupation VARCHAR, Annual_Income FLOAT)"
conn.execute(customers_ddl)
conn.register("customers_df", customers_table)
conn.execute("INSERT INTO customers SELECT * FROM customers_df")

# Accounts Table
accounts_ddl = "CREATE TABLE IF NOT EXISTS accounts (Customer_ID VARCHAR, Num_Bank_Accounts INTEGER, Num_Credit_Card INTEGER, Num_of_Loan INTEGER, Type_of_Loan VARCHAR)"
conn.execute(accounts_ddl)
conn.register("accounts_df", accounts_table)
conn.execute("INSERT INTO accounts SELECT * FROM accounts_df")

# Monthly Records Table
records_ddl = "CREATE TABLE IF NOT EXISTS monthly_records (ID VARCHAR, Customer_ID VARCHAR, Month VARCHAR, Monthly_Inhand_Salary FLOAT, Delay_from_due_date INTEGER, Num_of_Delayed_Payment INTEGER, Changed_Credit_Limit FLOAT, Num_Credit_Inquiries INTEGER, Credit_Mix VARCHAR, Outstanding_Debt FLOAT, Credit_Utilization_Ratio FLOAT, Credit_History_Age VARCHAR, Payment_of_Min_Amount VARCHAR, Total_EMI_per_month FLOAT, Amount_invested_monthly FLOAT, Payment_Behaviour VARCHAR, Monthly_Balance FLOAT)"
conn.execute(records_ddl)
conn.register("monthly_records_df", monthly_records_table)
conn.execute("INSERT INTO monthly_records SELECT * FROM monthly_records_df")

# Close the connection
conn.close()

Let's test the connection to the database and run a simple query to check if the data was loaded correctly.

In [ ]:
conn = duckdb.connect('credit_scoring.duckdb')
query_result = conn.execute("SELECT * FROM customers WHERE Age > 30").df()
conn.close()

query_result.head()

## Text to SQL pipeline
The next step is to build our Text to SQL pipeline. One of the easiest ways to do this is to leverage the [Vanna](https://github.com/vanna-ai/vanna) library. This library abstracts many of the patterns and best practices for building a Text to SQL pipeline and allows us to focus on the data and the model.

Behind the scenes, Vanna uses Retrieval Augmented Generation (RAG) to generate the SQL query. It utilizes a vector database to store information about our data such as the schema, table names, and column names, example SQL statements, data documentation, and others. This information is then used to retrieve the most relevant information when generating the SQL query.

Let's see how we can create a custom pipeline that leverages Bedrock and our DuckDB database.

In [ ]:
from vanna.chromadb import ChromaDB_VectorStore
from vanna.bedrock.bedrock_converse import Bedrock_Converse
import boto3
import re

Bellow is where we configure our Vanna text to sql pipeline. We'll use Claude 3 models from Bedrock, a local [ChromaDB](https://www.trychroma.com/) vector database that is natively supported by Vanna, and our DuckDB database.

There are a few modifications to get the best results with Claude3 models. First is the default `initial_prompt` is modified to instruct Claude to output the generated sql inot `<sql></sql>` xml tags. Secondly a custom `extract_sql` function is implemented to extract the generated sql from the xml tags.

In [ ]:
bedrock_runtime = boto3.client("bedrock-runtime")
config = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "temperature": 0.4,
    "max_tokens": 2048,
    "initial_prompt": f"You are an ANSI SQL expert.\n"
    + "Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions.\n"
    + "Always place the generated SQL into <sql></sql> tags.",
}


class MyVanna(ChromaDB_VectorStore, Bedrock_Converse):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Bedrock_Converse.__init__(self, client=bedrock_runtime, config=config)

    def extract_sql(self, llm_response: str) -> str:
        sql_text = re.search(r"<sql>(.*?)</sql>", llm_response, re.DOTALL)
        if sql_text:
            return sql_text.group(1)
        else:
            return llm_response


vn = MyVanna(config)

In [ ]:
# connect to the DuckDB database
vn.connect_to_duckdb(url='duckdb:///credit_scoring.duckdb')

### "Training" the RAG model
We can now begin to "train" the model by ingesting information into the ChromaDB vector database. First, Vanna can generate some basic documentation for the tables in the database using a built-in template. 

In [ ]:
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

In [ ]:
# train on the plan
vn.train(plan=plan)

In [ ]:
# Let's see what the generated documentation looks like
training_data = vn.get_training_data()
print(training_data.query("training_data_type == 'documentation'").iloc[0]["content"])

In [ ]:
training_data

Let's add a few more examples of training data using the DDL statements from earlier

In [ ]:
vn.train(ddl=customers_ddl)
vn.train(ddl=accounts_ddl)
vn.train(ddl=records_ddl)

We can add additional documentation as well. For example, the only thing the LLM would know about the `Month` field is that it is a `VARCHAR`, but the month could be represented in any number of ways for example `January`, `Jan`, `01`, `01/24`, etc. We can add this information to the vector database to help the LLM generate the correct SQL query when it needs to filter by month.

In [ ]:
vn.train(documentation="In the records table, the column Month has the following values: January, February, March, April, May, June, July, August")

In [ ]:
training_data = vn.get_training_data()
training_data

### Query the database with natural language

In [ ]:
# Let's start with a simple question
sql, result, _ = vn.ask(question="What is the average age of the customers?", visualize=False)
result

In [ ]:
# Average age seems high, let's find out why
sql, result, _ = vn.ask(question="Who are the top 5 oldest customers?", visualize=False)
result

In [ ]:
# Let's try another question
sql, result, _ = vn.ask(question="Which customer makes the largest average monthly payment?", visualize=False)
result 

In [ ]:
# This question would have failed if we did not provide the additional documentation about the values in the Month column
sql, result, _ = vn.ask(question="Which customer had the largest increase in monthly salary from 01 to 08? ", visualize=False, print_results=True)
result

In [ ]:
# let's try a more complex query
sql, result, _ = vn.ask(question="Create a binning of annual salary and analyze that against the average number of credit cards ", visualize=False, print_results=True)
result

## Conclusion
In this notebook, we have seen how to extract insights from structured data using Text to SQL. We have loaded a dataset into a database, created a Text to SQL pipeline using Vanna, and queried the database using natural language. This is a powerful tool that can be used to extract insights from structured data in a more intuitive way.